## Building a GPT

Following Andrej Karpathy video for building transformers . I will be using his notebook but will be typing the code myself, so that I learn what has been done in the notebook, and comment and improve the notebook in any way I can


This file is a script of the play "Coriolanus" by William Shakespeare.

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-09-25 04:14:48--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.02s   

2025-09-25 04:14:48 (59.2 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [ ]:
# Read the first few lines of the file
with open('input.txt','r') as f:
  content=f.read()
  print(content[:100])
  print(f'This is {len(content)} characters long')

# Count no of words
words=content.split()
print(f'This is {len(words)} words long')


First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You
This is 1115394 characters long
This is 202651 words long


In [ ]:
# here are all the unique characters that occur in this text
print(''.join(set(content)))
vocab_size=len(set(content))
unique_words=len(set(words))
print(f'The play has {vocab_size} characters and {unique_words} words')

wVJpnGWB&dv3L;-:FOqKNTRtYAc$gShHE?bXMoIu!PezQ.s'
flymr, CikaZxUjD
The play has 65 characters and 25670 words


In [ ]:
# create a mapping from characters to integers
# Need a dictionary which encodes english to integers
# Need a dictionary which takes those integers and spits out english

vocab=sorted(set(content))
encoded_lkp_tbl={char:i for i,char in enumerate(vocab)}
reverse_lkp_tbl={i:char for i,char in enumerate(vocab)}


def encode(s):
 encd_s=[encoded_lkp_tbl[char] for char in s]
 return (encd_s)

def decode(integer_sequence):
  decoded_s=[reverse_lkp_tbl[i] for i in integer_sequence]
  return ''.join(decoded_s)


print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [ ]:
#Rewriting the functions because I liked the way how lambda function is used
#in the original colab notebook
# and also creating two dictionaries is better than creating 1 dictionary
# because the logic for getting keys based on values isn't nice

# Archiving 2025
#encode = lambda e:[lkp_tbl[char] for char in e]
#print(encode("hii there"))


In [ ]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch
data=torch.tensor(encode(content))
print(data[:100])
data.shape,data.dtype

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


(torch.Size([1115394]), torch.int64)

In [ ]:
# Example
print(data[:9])
decode(data[:9].tolist())



tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])


'First Cit'

Why we cannot split the data randomly ?


In [ ]:
# Let's now split up the data into train and validation sets
# The author chose to partition the data based on length, however I am using sampling to split the data.
# The reason being I don't know the behavior of first 90% of the data versus the rest of the data

# We cannot use the below code. Train Test Split does shuffling and if we shuffle then sentences/words lose their meaning

#from sklearn.model_selection  import train_test_split
#train_data,val_data=train_test_split(data,test_size=0.1,random_state=40)


split=int(0.9*len(data))
train_data=data[:split]
val_data=data[split:]


# assert len(data) == len(train_data)+len(val_data)


In [ ]:
# Chose a window. Choosing a window is important because of attention mechanism.

#The notebook generates sequence character by character. The simplest way is to input the current token and based on
# current token generate the future token. Let's work toward this
block_size = 8  #window_size
sample=train_data[:block_size+1]   #+1 because we need the target for our entire block
for i in range(1,len(sample)):
    print(f'when the context is {sample[:i]} the target is {sample[i]}')


when the context is tensor([18]) the target is 47
when the context is tensor([18, 47]) the target is 56
when the context is tensor([18, 47, 56]) the target is 57
when the context is tensor([18, 47, 56, 57]) the target is 58
when the context is tensor([18, 47, 56, 57, 58]) the target is 1
when the context is tensor([18, 47, 56, 57, 58,  1]) the target is 15
when the context is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
when the context is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [ ]:
parallel_ct=4

# Using the minrex from above , let's try to process the dataset
# The logic is that the t'th token will output the t+1'th token
# Divide the entire dataset into batches of block sizes
# We will handle 4 input sequences having length equal to block size . Therefore, our batch is 4 rows deep and 8 columns wide (4,8)

#Convert the entire training data to tensors of dimensions batch size
sample=train_data[:32]
sample.reshape(4,8)

# The above reshaping would not work. Because for every block size(input of 8 char), our target is the 9th character
# If we try to shape the data then the inputs and target would coincide. Because the 9th character is part of second block.
# So, we have to reshape the data so that after every block, we leave a space for target


#So, for a batch of shape (4,8), we will have a target of shape (4,1). That is, for every input there is a corresponding target
sample=train_data[:32+4]

# create 1 batch
# Input of size block size
# Output of size block size

#initial
'''
start=0            tgt=1
end=7              end=8

start=9            tgt=10
end=16             end=17

start=18           tgt=19
end=25             end=26

start=27           tgt=28
end=34             end=35
'''

# Figuring out the start is needed


input=[]
tgt=[]

for i in range(parallel_ct):
  start=i*(block_size+1)
  end=start+(block_size)
  input.append(sample[start:end])
  tgt.append(sample[start+1:end+1])

print("Input :")
print(input)
print("Target :")
print(tgt)


#print(sample[:block_size])
#print(sample[block_size:block_size*2])




'''
for i in range(len(sample)):
  input=sample[:block_size+1]
  print('Input : ' ,input)
  tgt=sample[:block_size+1+i]
  print('Target :',tgt)
'''

Input :
[tensor([18, 47, 56, 57, 58,  1, 15, 47]), tensor([47, 64, 43, 52, 10,  0, 14, 43]), tensor([53, 56, 43,  1, 61, 43,  1, 54]), tensor([53, 41, 43, 43, 42,  1, 39, 52])]
Target :
[tensor([47, 56, 57, 58,  1, 15, 47, 58]), tensor([64, 43, 52, 10,  0, 14, 43, 44]), tensor([56, 43,  1, 61, 43,  1, 54, 56]), tensor([41, 43, 43, 42,  1, 39, 52, 63])]


"\nfor i in range(len(sample)):\n  input=sample[:block_size+1]\n  print('Input : ' ,input)\n  tgt=sample[:block_size+1+i]\n  print('Target :',tgt)\n"

The sample used above is a contiguous piece of text. But, since each sequence is independent of the other within a batch we can create sequences randomly within a batch. Therefore, we will need four random numbers between 0 and `len(content) - block_size `. For example if the content length is 20 and block size is 4

Generate inputs and targets for the model to learn

In [ ]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
n_embed=32

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")


inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(123)

class BigramLanguageModel(nn.Module):
  def __init__(self) -> None:
      super().__init__()
      self.token_embedding_table=nn.Embedding(vocab_size,n_embed)  #C = n_embed
      self.position_embedding_table= nn.Embedding(block_size,n_embed) #T=block_size
      self.lm_head =nn.Linear(n_embed,vocab_size)


  def forward(self,idx,targets=None):
      token_embeddings=self.token_embedding_table(idx)  # This returns as B*T*C
      positional_embeddings=self.position_embedding_table(torch.arange(block_size))
      x=token_embeddings+positional_embeddings
      logits=self.lm_head(x)
      if targets is None:
        loss=None
      else:
        B,T,C=logits.shape
        logits=logits.view(B*T,C)
        targets=targets.view(B*T)
        loss= F.cross_entropy(logits,targets)

      return logits,loss

  def generate(self,idx,max_new_tokens=100):

    for _ in range(max_new_tokens):
      logits,loss = self (idx) # refers to forward function
      logits=logits[:,-1,:] #extract the last timestep
      probs=F.softmax(logits,dim=-1)
      idx_next=torch.multinomial(probs,num_samples=1)
      idx=torch.cat((idx,idx_next),dim=1)

    return idx


m=BigramLanguageModel()

before_weights = m.token_embedding_table.weight.clone().detach()

logits,loss=m(xb,yb)
out=decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_tokens=500).tolist()[0])

after_weights = m.token_embedding_table.weight
print('Are the two weights equal ?',torch.equal(before_weights,after_weights))   # True No update in model


print('The output of the above token generator is \n',out)
print(f'\n Representation of inputs after passing through the embedding table {logits} and the loss is {loss}')

RuntimeError: The size of tensor a (2) must match the size of tensor b (8) at non-singleton dimension 1

In [ ]:
# Use optimizer to update the parameters

batch_size=4
grad_optim=torch.optim.AdamW(m.parameters(),lr=1e-3)
xb, yb = get_batch('train')

for _ in range(10000):
  logits,loss=m(xb,yb)
  grad_optim.zero_grad(set_to_none=True)
  loss.backward()
  grad_optim.step()

print(loss.item())


decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_tokens=500).tolist()[0])



0.08666006475687027


RuntimeError: The size of tensor a (2) must match the size of tensor b (8) at non-singleton dimension 1

In [ ]:
after_weights = m.token_embedding_table.weight


#Understanding matrix multiplication of tensors in pytorch


---


Matrix A: ( 2 * 3)

1    2    5

1    3    7


Matrix B: (3 * 2)

2   4

1   1

3   3


Matrix C = A*B  2*2

1x2 + 2x1 +5*3 = 19     1x4 + 2x1 + 5*3 = 21

1x2 + 3x1=5     1x4+3x1=7


## The mathematical trick in self-attention

In [ ]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [ ]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [ ]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [ ]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [ ]:
# Understanding the mathematical trick - Aadish
a = torch.tril(torch.ones(2,3))

a = a.masked_fill(a==0,float('-inf'))
a=F.softmax(a,dim=1)
print(a)
x.shape  # B,T,C 4,8,2    T,C 8,8



tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000]])


torch.Size([4, 8, 2])

In [ ]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
print(wei)

xbow3 = wei @ x
torch.allclose(xbow, xbow3)


tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


False

In [ ]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [ ]:
# Writing self-attention block

torch.manual_seed(1337)
batch_size = 4
block_size = 8
embed_size = 32

# tokenize

x,y=get_batch('train')  # batch, block ## block = Time dimension


head_size=16  # attention head . Positions that it will attend to .



embed=nn.Embedding(vocab_size,embed_size)


# Pass the input through the embeddings layer

x=embed(x)

# Create copies of input as query , key and vector

query=nn.Linear(embed_size,head_size,bias=False)
key=nn.Linear(embed_size,head_size,bias=False)
value=nn.Linear(embed_size,head_size,bias=False)

q=query(x)
k=key(x)
v=value(x)

#print(k.shape)  # B,T,head_size

# weights are the affinities between key and vector. Wherever the number is high
#the magnitude of weights reflect that.

wei=q @ k.transpose(-2,-1) #B,T,T


print(wei[0])

#print(wei.shape)  #B,T,T

tril=torch.tril(torch.ones(block_size,block_size))  #T * T
wei=wei.masked_fill(tril==0,float('-inf'))

print(wei[0])

wei=F.softmax(wei,dim=-1)

print(wei[0])

out= wei @ v
print(wei.shape)
print(v.shape)
print(out.shape)


tensor([[ 0.4034, -1.5845,  1.7946, -4.2395,  2.2851,  2.8202,  1.1968, -1.5845],
        [-0.2297,  2.1917,  2.4667, -0.2195, -0.3685, -1.0612, -0.5106,  2.1917],
        [ 0.2771,  0.3480, -0.3270,  0.4888, -0.6640, -0.7369, -0.1807,  0.3480],
        [ 2.4890,  0.1666,  1.3966, -1.3715,  2.3623,  1.1069, -0.3193,  0.1666],
        [ 0.2522,  0.4723, -0.4703,  0.5955,  0.6543,  0.2061,  0.8850,  0.4723],
        [ 0.7495, -2.4608, -0.6745, -1.9665, -0.5707,  1.0563,  0.4899, -2.4608],
        [-2.1033,  0.6029, -1.6932,  2.1043,  0.1394, -1.4633, -0.2959,  0.6029],
        [-0.2297,  2.1917,  2.4667, -0.2195, -0.3685, -1.0612, -0.5106,  2.1917]],
       grad_fn=<SelectBackward0>)
tensor([[ 0.4034,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-0.2297,  2.1917,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
        [ 0.2771,  0.3480, -0.3270,    -inf,    -inf,    -inf,    -inf,    -inf],
        [ 2.4890,  0.1666,  1.3966, -1.3715,    -inf,    -inf, 

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [ ]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [ ]:
k.var()

In [ ]:
q.var()

In [ ]:
wei.var()

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

In [ ]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

In [ ]:
module.parameters()

In [ ]:
# Implement LayerNorm - Aadish version

class GPTLayerNorm():
  def __init__(self,dim,eps=1e-5,momentum=0.1):
    self.eps=eps
    self.gamma=torch.ones(dim)
    self.beta=torch.zeros(dim)
  def __call__(self,dim,x):
    x_mean=x.mean(1,keepdim=True)
    x_var=x.var(1,keepdim=True)
    x_hat=self.gamma*((x-x_mean)/torch.sqrt(x_var+self.eps))+self.beta
    return x_hat
  def parameters(self):
    return ([self.gamma,self.beta])


n=GPTLayerNorm(100)
x=torch.randn(32,100)
y=n(100,x)

y[0,:].mean(), y[0,:].std()



(tensor(1.1921e-09), tensor(1.0000))

In [ ]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [ ]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

In [ ]:
# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>



### Full finished code, for reference

You may want to refer directly to the git repo instead though.

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.209729 M parameters
step 0: train loss 4.4116, val loss 4.4022
step 100: train loss 2.6568, val loss 2.6670
step 200: train loss 2.5090, val loss 2.5058
step 300: train loss 2.4194, val loss 2.4334
step 400: train loss 2.3501, val loss 2.3568
step 500: train loss 2.2963, val loss 2.3129
step 600: train loss 2.2410, val loss 2.2501
step 700: train loss 2.2057, val loss 2.2191
step 800: train loss 2.1633, val loss 2.1860
step 900: train loss 2.1242, val loss 2.1498
step 1000: train loss 2.1027, val loss 2.1298
step 1100: train loss 2.0692, val loss 2.1183
step 1200: train loss 2.0386, val loss 2.0797
step 1300: train loss 2.0276, val loss 2.0652
step 1400: train loss 1.9925, val loss 2.0370
step 1500: train loss 1.9702, val loss 2.0302
step 1600: train loss 1.9645, val loss 2.0487
step 1700: train loss 1.9421, val loss 2.0143
step 1800: train loss 1.9091, val loss 1.9953
step 1900: train loss 1.9085, val loss 1.9874
step 2000: train loss 1.8861, val loss 1.9957
step 2100: train loss 1.